In [1]:
import numpy as np
from skimage import exposure
#from skimage.segmentation import quickshift, slic, watershed
import geopandas as gpd
from osgeo import gdal, ogr
import scipy
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import skimage.feature as feature
import math

from hyperopt import hp,fmin,tpe,STATUS_OK,Trials

from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, cohen_kappa_score

import warnings
warnings.filterwarnings("ignore") 
from matplotlib import pyplot
from skimage.segmentation import slic, mark_boundaries, quickshift, watershed, felzenszwalb
##from sklearn import preprocessing
import rasterio
from skimage import io, color, segmentation
from rasterio import plot
from skimage.color import rgb2gray
from skimage.filters import sobel
from rasterio.plot import show

In [2]:
# Open the stacked band satellite data
dataset = gdal.Open(r"C:\Users\10333076966\Documents\GEOPROCESSAMENTO\VIDEIRA\TESTE_NOVO\CBERS_RGB_2m_.tif")
driverTiff = gdal.GetDriverByName('GTiff')

# Read the stacked bands into a NumPy array
img_stack = np.zeros((dataset.RasterCount, dataset.RasterYSize, dataset.RasterXSize), dtype=np.float32)
for i in range(dataset.RasterCount):
    band = dataset.GetRasterBand(i+1)
    img_stack[i,:,:] = band.ReadAsArray().astype(np.float32)
    band = None

# Convert stacked bands to RGB image
img_rgb = np.transpose(img_stack, (1,2,0))

# Convert RGB image to HSV color space
img_hsv = color.rgb2hsv(img_rgb)

# Perform SLIC segmentation on HSV image
#segments = slic(img_hsv, n_segments=30000, compactness=5, sigma = 0.001)



In [114]:
img_lab = color.rgb2lab(img_rgb)
segments = slic(img_lab, n_segments=40000, compactness = 3, sigma = 0.001)

In [47]:
gradient = sobel(rgb2gray(img_rgb))
segments = watershed(gradient, markers=8000, compactness=0.001)

In [3]:
segments = felzenszwalb(img_rgb, scale=400, sigma=1.5, min_size=100, channel_axis=2)

In [4]:
clfds = driverTiff.Create(r'C:\Users\10333076966\Documents\GEOPROCESSAMENTO\VIDEIRA\TESTE_NOVO\SEGMENT_FELZEN.tif', dataset.RasterXSize, dataset.RasterYSize,
                          1, gdal.GDT_Int32)
clfds.SetGeoTransform(dataset.GetGeoTransform())
clfds.SetProjection(dataset.GetProjection())
clfds.GetRasterBand(1).SetNoDataValue(-9999.0)
clfds.GetRasterBand(1).WriteArray(segments)
clfds = None